# Testing random number generation

This is an inherently iterative process so we won't vectorize to begin with, to show how these work in a simpler way. 

We will investigate a "poor" generator that repeats quickly, a more reasonable Park-Miller scheme, and the default `python` scheme from the `random` library. 

In [ ]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt

In [ ]:

poor_seed = 123                         # maintain seed for poor generator
def poor():                             # example from Numerical Recipes
    global poor_seed
    seed = poor_seed
    IM = 6075 ; IA = 106 ; IC = 1283
    seed = (IA * seed + IC) % IM        # linear congruential algorithm
    poor_seed = seed
    return seed / float(IM)

park_miller_seed = 1                    # maintain seed for Park-Miller
def park_miller():                      # Park-Miller generator
    global park_miller_seed
    seed = park_miller_seed
    IA = 16807 ; IC = 2147483647 ; IQ = 127773 ; IR = 2836
    h = seed // IQ
    l = seed % IQ;
    seed = IA * l - IR * h
    if seed <= 0:
        seed += IC
    park_miller_seed = seed
    return seed / float(IC)

def test(generator, name, remark, eyeballtest):
    print (" " + name + "() " + remark)

    print (" checking period ...")
    start = generator()
    max_steps = 1000000
    for i in range(max_steps):
        if generator() == start:
            print (" repeats after " + repr(i) + " steps")
            break
    if i+1 >= max_steps:
        print (" period larger than " + repr(max_steps))

    start_time = time.perf_counter()
    bins = 10000
    print (" binning " + repr(max_steps) + " tries in " + repr(bins) + " bins")
    bin = np.zeros(bins)
    for i in range(max_steps):
        b = int(generator() * bins)
        if b >= 0 and b < bins:
            bin[b] += 1
    expect = max_steps / float(bins)
    chisqr = np.sum( (bin - expect)**2 ) / expect
    cpu_time = time.perf_counter() - start_time
    print (" chi-squar/d.o.f. = " + repr(chisqr/(bins-1.0)))
    print (" CPU time = " + repr(cpu_time) + " seconds")

    for i in range( len(eyeballtest) ):
        eyeballtest[i] = [generator(), generator()]



## Testing random number generators

In [ ]:
poor_pairs = np.zeros( (100000, 2) )
park_miller_pairs = np.zeros( (100000, 2) )
py_pairs = np.zeros( (100000, 2) )
test(poor, "poor", "from Numerical Recipes period 6,075", poor_pairs)
print("----")
test(park_miller, "park_miller", "Park-Miller multiplicative generator", park_miller_pairs)
print("----")
test(random.random, "random.random", "from Python standard library", py_pairs)

## "Eyeball" test

Now we plot the generator sequence as pairs on a scatter plot. Bad generators are immediately discernible due to their obvious correlations. 

In [ ]:
f1 = plt.figure(1, figsize=(13,13))
plt.plot(poor_pairs[:,0], poor_pairs[:,1],'b.')
plt.show()

In [ ]:
f2 = plt.figure(2, figsize=(13,13))
plt.plot(park_miller_pairs[:,0], park_miller_pairs[:,1],'r.')
plt.show()

In [ ]:
f3 = plt.figure(3, figsize=(13,13))
plt.plot(py_pairs[:,0], py_pairs[:,1],'g.')
plt.show()